# Immoweb: WebScraping_03

In [1]:
import re
import pickle
import requests
from bs4 import BeautifulSoup
import unidecode
import pandas as pd
from time import time, sleep
from IPython.display import clear_output
from random import randint
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webscraping_functions import *

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
regions = ['anvers', 'limbourg', 'flandre-occidentale', 'flandre-orientale', 'hainaut', 'bruxelles',\
             'luxembourg','liege','namur','brabant-flamand','brabant-wallon']

In [3]:
type_of_goods = ['appartement','immeuble-de-rapport']

#### Testing the url concatenation 

https://www.immoweb.be/robots.txt

https://www.immoweb.be/fr/annonce/immeuble-a-appartements/a-vendre/tamines/5060/8616521?searchId=5f63145e1657d

In [4]:
help(string_preprocessing)

Help on function string_preprocessing in module webscraping_functions:

string_preprocessing(feature_name)
    Return preprocessed feature_name
    
    :param feature_name (string): name of feature that has to be preprocessed
    :return (string): preprocessed name of feature (feature_name)



### 1. Scraping Steps For Immoweb
#### 1. Get the full url

In [5]:
help(return_full_url)

Help on function return_full_url in module webscraping_functions:

return_full_url(type_of_good, region, page_num=1, for_sale=True)
    Returns the immoweb url with regards to type_of_good, region and page_num
    
    :param type_of_good (string): 'appartement' or 'immeuble-de-rapport'
    :param region (string): one of the belgian provinces written in french
    :param page_num (int): page number of websearch
    :param for_sale (boolean): True for properties for sales, False for properties to rent
    :return:



In [6]:
# Get url
full_url = return_full_url('immeuble-de-rapport', 'namur', 1)
print(full_url)

https://www.immoweb.be/fr/recherche/immeuble-de-rapport/a-vendre/namur/province?countries=BE&page=1&orderBy=relevance


In [9]:
full_url_rent = return_full_url('appartement', 'namur', 1, for_sale=False)
print(full_url_rent)

https://www.immoweb.be/fr/recherche/appartement/a-louer/namur/province?countries=BE&page=1&orderBy=relevance


#### 2. Parsing Search Page related to region and type of good 

In [4]:
help(parsed_protected_page)

Help on function parsed_protected_page in module webscraping_functions:

parsed_protected_page(url)
    Returns the parsed page through BeautifulSoup with Chrome as webdriver
    
    :param url (string): url of that we want to parse
    :return: the parsed page through BeautifulSoup with Chrome as webdriver



In [5]:
scraped_search_page = parsed_protected_page(full_url)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jeromedharveng/.wdm/drivers/chromedriver/mac64/87.0.4280.20/chromedriver] found in cache


In [ ]:
scraped_search_page

#### 3. Get number of pages and individual urls from scraped page

In [11]:
help(get_nb_pages)

Help on function get_nb_pages in module webscraping_functions:

get_nb_pages(url)
    Returns the total number of search pages resulting of the search query
    
    :param url (string):
    :return: the total number of search pages resulting of the search query



In [12]:
help(get_individual_urls)

Help on function get_individual_urls in module webscraping_functions:

get_individual_urls(one_search_page_url, filter_opt=True)
    Returns 3 lists, with the urls, the postcodes and cities
    
    :param one_search_page_url (string): url of 1 single search page
    :param filter_opt (boolean): if True => filter out the offers for several goods (for example 10 apts)
    :return: 4 lists, with the urls, the prices, the postcodes and cities



##### Test  Properties For Sale

In [6]:
nb_pages = get_nb_pages(full_url)
print(nb_pages)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/jeromedharveng/.wdm/drivers/chromedriver/mac64/87.0.4280.20/chromedriver] found in cache


 
10


In [14]:
list_urls, price_list, list_postcodes, list_cities = get_individual_urls(full_url,False)

In [15]:
print(len(list_urls))
print(list_urls[:5])

30
['https://www.immoweb.be/fr/annonce/immeuble-a-appartements/a-vendre/jambes/5100/8788132?searchId=5fb6931c0c820', 'https://www.immoweb.be/fr/annonce/immeuble-industriel-mixte/a-vendre/sambreville/5060/8806781?searchId=5fb6931c0c820', 'https://www.immoweb.be/fr/annonce/immeuble-mixte/a-vendre/spy/5190/9000608?searchId=5fb6931c0c820', 'https://www.immoweb.be/fr/annonce/immeuble-de-bureaux/a-vendre/namur/5000/8373299?searchId=5fb6931c0c820', 'https://www.immoweb.be/fr/annonce/immeuble-a-appartements/a-vendre/saint-servais/5002/8958297?searchId=5fb6931c0c820']


In [16]:
print('Length of postcodes : ', len(list_postcodes))
print(list_postcodes[:5])
print('Length of prices list : ', len(price_list))
print(price_list[:5])
print('Length of cities : ', len(list_cities))
print(list_cities[:5])

Length of postcodes :  30
['5100', '5060', '5190', '5000', '5002']
Length of prices list :  30
['395000', '800000', '625000', '1450000', '300000']
Length of cities :  30
['Jambes', 'Sambreville', 'Spy', 'Namur', 'Saint-servais']


##### Test Properties For Rent

In [17]:
list_urls_rent, price_list_rent, list_postcodes_rent, list_cities_rent = get_individual_urls(full_url_rent,False)

In [18]:
print(len(list_urls_rent))
print(list_urls_rent[:5])

30
['https://www.immoweb.be/fr/annonce/appartement/a-louer/gembloux/5030/9041391?searchId=5fb69324c3158', 'https://www.immoweb.be/fr/annonce/duplex/a-louer/suarlee/5020/9028087?searchId=5fb69324c3158', 'https://www.immoweb.be/fr/annonce/appartement/a-louer/namur/5000/9035029?searchId=5fb69324c3158', 'https://www.immoweb.be/fr/annonce/appartement/a-louer/namur/5000/9038812?searchId=5fb69324c3158', 'https://www.immoweb.be/fr/annonce/appartement/a-louer/jambes/5100/8940663?searchId=5fb69324c3158']


In [19]:
print('Length of postcodes : ', len(list_postcodes_rent))
print(list_postcodes_rent[:5])
print('Length of prices list : ', len(price_list_rent))
print(price_list_rent[:5])
print('Length of cities : ', len(list_cities_rent))
print(list_cities_rent[:5])

Length of postcodes :  30
['5030', '5020', '5000', '5000', '5100']
Length of prices list :  30
['80', '60', '885', '120', '90']
Length of cities :  30
['Gembloux', 'Suarlée', 'Namur', 'Namur', 'Jambes']


**Comments:**
> - Regarding the prices, we observe in some cases it's not optimal for example if displayed rentald price is 690 € (+ 80 €).
> - Then currently the scraped price at this point will be 80
> - Let's see what the price scraped with get_features_one_page gives....

#### 4. Get the features of the found urls

##### A. From a single page

In [20]:
help(get_features_one_page)

Help on function get_features_one_page in module webscraping_functions:

get_features_one_page(url, postcode, city, price, region, type_of_good)
    Returns dictionnary with the different house features of current immoweb page
    
    :param url (string): url of property we want to scrape the features from
    :param postcode (string): postcode gathered through get_individual_urls()
    :param city (string): city gathered through get_individual_urls()
    :param price (string): price gathered through get_individual_urls()
    :param region (string): region know from  return_full_url
    :param type_of_good (string): 'appartement' or 'immeuble-de-rapport'
    :return (dict): dictionnary containing the different house features of the current immoweb page



##### Test Properties For Sales

In [21]:
features_1 = get_features_one_page(list_urls[0], list_postcodes[0], list_cities[0],\
                                   price_list[0],'namur', 'immeuble-de-rapport')

In [22]:
features_1

{'page_url': 'https://www.immoweb.be/fr/annonce/immeuble-a-appartements/a-vendre/jambes/5100/8788132?searchId=5fb6931c0c820',
 'postcode': '5100',
 'city': 'Jambes',
 'type_of_good': 'immeuble-de-rapport',
 'price': '395000',
 'region': 'namur',
 'immoweb_code': '8788132',
 'street_property': "Rue d'Enhaive",
 'street_number_property': '125',
 'description': 'Maison de rapport principalement destinée pour investisseurs, professions libérales, Sociétés, Restaurateurs, Banques..Composée de 3 étages hors caves comprenant 2 logements avec possibilités de kots ou appartement supplémentaire. 3 compteurs gaz neufs séparés, 3 compteurs électriques séparés, 1 compteur eau.Situation géographique exceptionnelle, accès autoroutes, commerces à proximités, Bâtiments région wallonne à proximité. Transports en commun vers toutes directions. Gare SNCB Jambes.  Centre ADEPS. Cadre agréable et calme tout en jouissant des avantages décrits ci-dessus. Prix négociable',
 'picture_url': 'https://static.immow

##### Test Properties For Rent

In [27]:
features_1_rent = get_features_one_page(list_urls_rent[0], list_postcodes_rent[0], list_cities_rent[0],\
                                   price_list_rent[0],'namur', 'appartement')

In [28]:
features_1_rent

{'page_url': 'https://www.immoweb.be/fr/annonce/appartement/a-louer/gembloux/5030/9041391?searchId=5fb69324c3158',
 'postcode': '5030',
 'city': 'Gembloux',
 'type_of_good': 'appartement',
 'price': '80',
 'region': 'namur',
 'immoweb_code': '9041391',
 'street_property': "Rue de L'Escaille",
 'street_number_property': '31',
 'description': "Magnifique appartement situé au 3eme étage d’un bel immeuble soigné !Appartement basse énergie (PEB B), avec une très belle terrasse carrelée de 10m2, une place de parking intérieur + un grenier de 11 m2.Le tout dans un état irréprochable.Petit hall avec toilette séparée, une chambre (placard intégré) donnant accès à la salle de bain.Sdb baignoire avec paroi de douche et un lave-linge. Depuis le hall, une porte en verre donne accès à l’espace de vie, cuisine équipée, frigo américain, living salle à manger donnant sur la terrasse orientée plein sud.Luminaires et tentures & rideaux déjà placés ! Un environnement verdoyant et agréable.A 1km  de la gar

#### B. Scrape all features from 1 search page

In [30]:
def scrape_features_search_urls_1page(type_of_good, region,nb_page=1):
    # Get url of immoweb for first Search page
    full_url = return_full_url(type_of_good, region, nb_page)
    # Get list of result urls of that page
    urls, prices, postcodes, cities = get_individual_urls(full_url)
    
    ## Get the features of the different found urls
    feature_list = []
    
    for url, price, postcode, city in zip(urls, prices, postcodes, cities):
        features = get_features_one_page(url, postcode, city,\
                                   price, region, type_of_good)
        feature_list.append(features)
        
    
    return feature_list
    
    

In [29]:
feature_list = scrape_features_search_urls_1page('immeuble-de-rapport','namur')

NameError: name 'scrape_features_search_urls_1page' is not defined

In [27]:
len(feature_list)

30

In [28]:
feature_list[-1]

{'postcode': '5650',
 'city': 'Walcourt',
 'type_of_good': 'immeuble-de-rapport',
 'price': '120000',
 'region': 'namur',
 'immoweb_code': '8621235',
 'description': "WALCOURT - Rue de la station 79: Au centre du village, bon immeuble avec rez commercial de +/- 40m², actuellement utilisé comme librairie (possibilité de reprendre le fonds de commerce) et 75m² aménageables répartis sur 2 niveaux comprenant: Rez commercial de +/- 40m², réserve, wc séparé et chaufferie. Etage: possibilité de créer un appartement sur 2 niveaux de +/- 75m². Commodités: Châssis DV en PVC de 2015, chauffage par pompe à chaleur au rez et central mazout pour l'étage, 2 compteurs électriques (bi-horaire au rez), plate forme en roofing en parfait état, volets (électriques pour la vitrine), système d'alarme, jardin surélevé. Très bonne situation au centre du village, possibilité de reprendre le fonds de commerce. PEB F - 507 Kwh/m²/an - CU: 20110928011931. Prix: FAIRE OFFRE A PARTIR DE 120.000 € (sous réserve d'acc

#### C. Scrape all features for several pages

In [4]:
help(scraping_diff_regions)

Help on function scraping_diff_regions in module webscraping_functions:

scraping_diff_regions(type_of_good, list_of_regions, list_codes_already_scraped)
    Returns a list of dictionnaries with the individual house features
    :param type_of_good (string): 'appartement' or 'immeuble-de-rapport'
    :param list_of_regions (string): list of regions that have to be scraped
    :param list_immoweb_code_already_done (string): list of codes of houses that have already been scraped before
        in order to avoid scraping them again.
    :return (list): list of dictionnaries with the individual house features



##### Test Properties For Sale

In [5]:
code_already_scraped = ['7921691', '8185835', '8774561', '8691144', '8946649', '8737062', '8909716', '8707202', '7926099', '8452068', '8986115', '8371050', '8853665', '8888090', '8996778', '8822554', '8767775', '8766378', '7801613', '8773697', '8958297', '8889936', '8779618', '8815413', '8725629', '8459077', '8937005', '8698003', '8817865', '8802134', '8557828', '8455243', '8973719', '8891748', '8926773', '8976235', '8966056', '8999022', '8917282', '8927890', '8965746', '8808666', '8971103', '8880916', '8983822', '8276670', '8849545', '8684334', '8616796', '8116438', '8301394', '8549954', '8781788', '8953119', '7464828', '8981663', '8580208', '8893453', '8999092', '8867407', '8642620', '8794105', '8504109', '8883723', '8977514', '8744028', '8925673', '8815415', '8986050', '8859444', '8344391', '8948944', '7846451', '8609728', '8867406', '8818732', '7807928', '8817046', '8994246', '8958340', '8760414', '8971074', '8973720', '8684335', '7858110', '8953933', '8629040', '8971097', '8996171', '8847526', '7823589', '8261171', '8717537', '8946848', '8870465', '8643942', '8946388', '8813288', '8598447', '8928055', '8744217', '8914971', '8726126', '8884306', '8621235', '8778941', '8971078', '8373299', '8406290', '8934459', '8289340', '8971182', '8107658', '8904245', '8560797', '8969192', '8809575', '8938707', '8788132', '8714772', '8746195', '8860360', '8275740', '8916631', '8662278', '8698500', '8964119', '8378127', '8808667', '8887290', '8446056', '8979676', '8083497', '8876491', '8925114', '8804935', '8915448', '8711888', '8941881', '8082983', '8704521', '8886061', '8985489', '8815417', '8654634', '8978776', '8754975', '8442405', '8774677', '8791029', '8956844', '8983571', '8530413', '8927889', '8870631', '8813286', '8782034', '8946850', '8932066', '8964219', '8957281', '8944001', '8691010', '8957163', '8935021', '6065502', '8709923', '8804470', '8927118', '8939926', '8967491', '8747785', '8895492', '8918866', '8616258', '8930172', '8922142', '8870006', '8849685', '7921690', '8210457', '8929499', '8704475', '8911679', '8945064', '8534593', '8847280', '8921779', '8554344', '7982981', '8962886', '8747786', '8986076', '7687057', '8894251', '8927165', '8913299', '8991913', '8568264', '8949601', '8899461', '8455051', '8868694', '8873798', '8884271', '8538364', '7423018', '8882567', '8808386', '8767834', '8922001', '8843978', '8870683', '8990860', '8795332', '8875369', '8764916', '8936952', '8851353', '8953934', '7863782', '8960597', '8806781', '8828633', '8782037', '8853027', '8924152', '8940338', '7838783', '8784270', '8748338', '8215224', '8986640', '8591927', '8724955', '8572228', '8701245', '8917854', '8815414', '8787707', '8998924', '8969140', '8223554', '8989581', '8990812', '8950869', '8546677', '8559851', '8983700', '8095049', '8882396', '8732555', '8942920', '8961366', '8950415', '8904790', '8851666', '8859128', '8648161', '8550617', '8823602', '8958510', '8932435', '8626251', '8717806', '8615564', '8824738', '8813285', '8654460', '7550917', '8714771', '8707353', '8940339', '8950414', '7360595', '8971166', '8407193', '8633644', '8871652', '8984679', '8986326', '8813287', '7653442', '8980686', '8897162', '8945051', '8691145', '8810999', '8442233', '8859127']

##### Test Properties For Rent

##### Scraping for Buildings and Flats

In [30]:
list_of_features = scraping_diff_regions('immeuble-de-rapport', regions, [])

The Scraping action for the region of liege has taken 0.0h-42.0min


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 85


In [5]:
apt_list_of_features = scraping_diff_regions('appartement', regions, [])

The Scraping action for the region of brabant-wallon has taken 1.0h-42.0min
The Total Scraping action has taken 19.0h-7.0min


### 2. Testing Scraping immoweb but with list of good already scraped

#### a. Loading previous (concatenated) df 

In [7]:
##Loading the different scraped dataframes
with open('./Saved_Variables/20201021_all_regions_immeuble-de-rapport_features.pkl','rb') as f:
    apt_building_df = pickle.load(f)

In [8]:
with open('./Saved_Variables/20201021_all_regions_appartement_features.pkl','rb') as f:
    apartments_df = pickle.load(f)

#### b. Extracting the list of immoweb_codes for goods already scraped

In [9]:
codes_building_scraped = list(apt_building_df['immoweb_code'])

In [10]:
codes_apartments_scraped = list(apartments_df['immoweb_code'])

In [11]:
len(codes_building_scraped), len(codes_apartments_scraped)

(4555, 7822)

#### c. Scraping Immoweb but taking into account the lists of codes already scraped

In [12]:
list_of_features = scraping_diff_regions('immeuble-de-rapport', regions, codes_building_scraped)

The Scraping action for the region of brabant-wallon has taken 0.0h-2.0min
The Total Scraping action has taken 1.0h-14.0min


In [13]:
apt_list_of_features = scraping_diff_regions('appartement', regions, codes_apartments_scraped)

The Scraping action for the region of brabant-wallon has taken 0.0h-12.0min
The Total Scraping action has taken 2.0h-24.0min


In [18]:
soup

<html class="no-js" lang="en-US"><!--<![endif]--><head>
<title>Attention Required! | Cloudflare</title>
<meta id="captcha-bypass" name="captcha-bypass"/>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" media="screen,projection" rel="stylesheet" type="text/css"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" type="text/css" media="screen,projection" /><![endif]-->
<style type="text/css">body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function () {
      var cookieEl = document.getElementById('cookie-alert');
      cookieEl.style.display = 'block';


In [16]:
url = "https://www.immoweb.be/fr/recherche/immeuble-de-rapport/a-vendre/anvers/province?countries=BE&page=1&orderBy=relevance"

### 3. Scraping Rentals for Appartments

## Realo

source: https://www.realo.be/fr/villes

In [7]:
# Return parsed web page
def parsed_page(url):
    response = requests.get(url)
    
    # Send a warning if Response code isn't 200
    #if response.status_code != 200:
    #    warn(f'Request for url : {url} has code: {response.status_code}')
    content = response.content
    parser = BeautifulSoup(content, 'html.parser')
    return parser

In [8]:
realo_base_url = "https://www.realo.be"
realo_cities_url = "https://www.realo.be/fr/villes"

In [9]:
def get_realo_cities_links(realo_cities_url):
    realo_base_url = "https://www.realo.be"
    
    # Parse realo's city's page
    realo_soup = parsed_page(realo_cities_url)
    
    realo_list = realo_soup.select(".icn-after-arrow-right")
    
    realo_dict = {} # Dictionnary to save scraped cities and respective links
    for city in realo_list:
        city_name = city.get_text(strip=True)
        url_city = city.get('href')
        realo_dict[city_name] = realo_base_url + url_city
    
    return realo_dict

In [10]:
realo_dict = get_realo_cities_links(realo_cities_url)

In [11]:
realo_dict

{"'s-Gravenvoeren": "https://www.realo.be/fr/3798-'s-gravenvoeren/5195912",
 "'s-Gravenwezel": "https://www.realo.be/fr/2970-'s-gravenwezel/5193893",
 "'s-Herenelderen": "https://www.realo.be/fr/3700-'s-herenelderen/5195899",
 'Aaigem': 'https://www.realo.be/fr/9420-aaigem/5194687',
 'Aalbeke': 'https://www.realo.be/fr/8511-aalbeke/5194495',
 'Aalst (9300)': 'https://www.realo.be/fr/9300-aalst/5194612',
 'Aalst (3800)': 'https://www.realo.be/fr/3800-aalst/5195769',
 'Aalter': 'https://www.realo.be/fr/9880-aalter/5194737',
 'Aarschot': 'https://www.realo.be/fr/3200-aarschot/5194125',
 'Aarsele': 'https://www.realo.be/fr/8700-aarsele/5194576',
 'Aartrijke': 'https://www.realo.be/fr/8211-aartrijke/5194400',
 'Aartselaar': 'https://www.realo.be/fr/2630-aartselaar/5193855',
 'Abolens': 'https://www.realo.be/fr/4280-abolens/5195670',
 'Abée': 'https://www.realo.be/fr/4557-abee/5195425',
 'Achel': 'https://www.realo.be/fr/3930-achel/5195803',
 'Achet': 'https://www.realo.be/fr/5362-achet/5196

In [12]:
one_page_realo = parsed_protected_page("https://www.realo.be/fr/1981-hofstade/5194105")

In [13]:
one_page_realo.select(".component-neighbourhood-info-lists")

[<div class="component-neighbourhood-info-lists loading" data-scope="componentNeighbourhoodInfoLists">
 <div class="loader">
 <div class="container">
 <div class="content">
 				Chargement des résultats
 			</div>
 </div>
 </div>
 <div class="error">
 		Erreur lors du chargement des données.
 	</div>
 <ul class="list-unstyled">
 <li class="list-properties hidden" data-ajax="/address/5194105/properties-around-me.json" data-scope="listProperties" data-template="components/address/neighbourhood_info_lists_properties">
 </li>
 <li class="list-local-info" data-ajax="/address/5194105/local-info.json" data-scope="listLocalInfo" data-template="components/address/neighbourhood_info_lists_local_info">
 </li>
 <li class="list-district-info" data-ajax="/address/5194105/district-info.json" data-scope="listDistrictInfo" data-template="components/address/neighbourhood_info_lists_district_info">
 </li>
 </ul>
 </div>]

In [253]:
one_page_realo.select(".component-neighbourhood-info-lists-district-info")

[]

def parsed_protected_realo_page(url):
    # Use the headless option to avoid opening a new browser window
    # source: https://towardsdatascience.com/web-scraping-with-selenium-d7b6d8d3265a
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    desired_capabilities = options.to_capabilities()
    driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver",desired_capabilities=desired_capabilities)
    
    # Get Page with Url
    get_url = driver.get(url)
    
    # Implicit wait
    #driver.implicitly_wait(10)
    # Explicit wait
    #wait = WebDriverWait(driver, 20)
    #wait.until(EC.presence_of_element_located((By.CLASS_NAME, "component-neighbourhood-info-lists-district-info")))

    soup = BeautifulSoup(driver.page_source,'html.parser')
    driver.quit()
    return  soup

In [263]:
one_page_realo = parsed_protected_rea_page("https://www.realo.be/fr/1981-hofstade/5194105")

In [258]:
one_page_realo.select(".component-neighbourhood-info-lists-district-info")

[]

In [264]:
one_page_realo

<html class="lang-fr country-be placeholders js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths js flexbox flexboxlegacy canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths browser-chrome" lang=